Log in as a standard user. List all the tracks in the database along with some of their
information. Filter the tracks by their artists and genres. Select a track and press “Play”.
Then go back to the menu and this time search for albums. Select an album and from the
album select a track to play. Then go to your user history page to see the last tracks you
have listened to. Log out and log in as one of the artists that the standard user listened to.
Check how many times the track has been played in the last 1 month.


In [5]:
import sqlite3
import PySimpleGUI as sg
import datetime
from datetime import datetime, timedelta


con = sqlite3.connect('Stage 3-2.db')
cur = con.cursor()

def Login_As():
    layout = [
        [sg.Text('Login as', size=(10))],
        [sg.Button('Artist', size=(10))],
        [sg.Button('Standard', size=(10))],
        [sg.Exit()]
    ]
    
    window = sg.Window('Main Window', layout)
    while True:
        event, values = window.read()

        if event == sg.WINDOW_CLOSED or event == 'Exit':
            break
        if event == 'Artist':
            window.close()
            Login_Artist()

        if event == 'Standard':
            window.close()
            Login_Standard()
    
    window.close()

def Login_Standard():
    layout = [
        [sg.Text('Username:')],
        [sg.Input(key='username_input')],
        [sg.Text('Password:')],
        [sg.Input(key='password_input',password_char='*')],
        [sg.Button('Enter'), sg.Button('Back'), sg.Exit()]
    ]

    window = sg.Window('Standard User Login Page', layout)
    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, 'Exit'):
            break
        if event == "Enter":
            username = values['username_input']
            password = values['password_input']
            if username == "":
                sg.popup("Please enter your username!")
            elif password == "":
                sg.popup("Please enter your password!")
            else:
                cur.execute('SELECT Username FROM User WHERE Username IN (SELECT Username FROM Standard) AND Username = ? AND Password = ?', (username, password))
                row = cur.fetchone()
                if row is not None:
                    sg.popup_timed("Login Succesful", non_blocking=True, no_titlebar=True, auto_close_duration=1, keep_on_top = True)
                    sg.time.sleep(1)
                    window.close()
                    
                    Main_Page(username)
                else:
                    sg.popup("Wrong username or password!")
                
        elif event == "Back":
            window.close()
            Login_As()

            
    window.close()

def Login_Artist():
    layout = [
        [sg.Text('Username:')],
        [sg.Input(key='username_input')],
        [sg.Text('Password:')],
        [sg.Input(key='password_input',password_char='*')],
        [sg.Button('Enter'), sg.Button('Back'), sg.Exit()]
    ]
    
    window = sg.Window('Artist Login Page', layout)
    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, 'Exit'):
            break
        if event == "Enter":
            username = values['username_input']
            password = values['password_input']
            if username == "":
                sg.popup("Please enter your username!")
            elif password == "":
                sg.popup("Please enter your password!")
            else:
                cur.execute('SELECT Username FROM User WHERE Username IN (SELECT Username FROM Artist) AND Username = ? AND Password = ?', (username, password))
                row = cur.fetchone()
                
                #Bu alttaki if kısmından artist arayüzü oluşturulacak!!
                if row is not None:
                    sg.popup_timed("Login Successful", non_blocking=True, no_titlebar=True, auto_close_duration=1, keep_on_top=True)
                    sg.time.sleep(1)
                    window.close()
                    Artist_Page(username)
                else:
                    sg.popup("Wrong username or password!")
        elif event == "Back":
            window.close()
            Login_As()

    window.close()

def Main_Page(username):
    layout = [
        [sg.Button('My Profile',size=(15,1))],
        [sg.Button('Tracks',size=(15,1))],
        [sg.Button('Playlists',size=(15,1))],
        [sg.Button('Albums',size=(15,1))],
        [sg.Button('History',size=(15,1))],
        [sg.Button('Search Users',size=(15,1))],
        [sg.Button('Logout',size=(15,1))],
     
    ]
    
    window = sg.Window('Main Page', layout)
    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED,): 
            #No exit function defined this time.
            break
        if event == "My Profile":
            window.close()
            Open_Profile(username)    
        if event == "Tracks":
            window.close()
            Tracks(username)
        elif event == "Playlists":
            window.close()
            Playlists(username)
        elif event == "Albums":
            window.close()
            Albums(username)
        elif event == "Search Users":
            window.close()
            Search_Users(username)
        elif event == "History":
            window.close()
            History(username)
        elif event == "Logout":
            window.close()
            Login_As()

    window.close()   
    

history_P=[]
followee=[]

def Tracks(UserName):
    # Get all tracks
    tracks = []
    
    for row in cur.execute('SELECT T_Name, Username, Genre, Length, T.T_ID FROM Track T, Create_T C WHERE T.T_ID = C.T_ID'):
                t_name = row[0]
                genre = row[1]
                length = row[2]
                username = row[3]
                t_id = row[4]
                tracks.append((t_name,genre,length,username))
    # Initialize selected tracks and filter variables
    selected_tracks = tracks
    artists = ['All']
    genres = ['All']
    
    # Get unique artists and genres
    for row in cur.execute('SELECT Username FROM Artist'):
        if row[0] not in artists:
            artists.append(row[0])
    
    for row in cur.execute('SELECT Genre FROM Track'):
        if row[0] not in genres:
            genres.append(row[0])
    
    # Create the layout for the GUI window
    layout = [
        [sg.Text('Artist:', size=(10, 1)), sg.Combo(artists, default_value='All', key='artist'),
         sg.Text('Genres:', size=(10, 1)), sg.Combo(genres, default_value='All', key='genre'), sg.Button(('Filter'), size=(17, 1))],
        [sg.Text("Track Names", pad=((10, 0), 10)), sg.Text("Artist", pad=((10, 0), 10)), sg.Text("Genre", pad=((10, 0), 10)), sg.Text("Length", pad=((10, 70), 10))
         ,sg.Button("Add to Playlist",size=(24,1))],
        [sg.Listbox(values=selected_tracks, size=(40, 7), key='track'), sg.Button('Play', size=(14, 7)),
         sg.Button('Back to Menu', size=(8, 7))]
    ]
    
    # Create the window
    window = sg.Window('Track List Window', layout)
    
    while True:
        event, values = window.read()
    
        if event == sg.WINDOW_CLOSED:
            break
    
        if event == 'Filter':
            username = values['artist']
            genre = values['genre']
            selected_tracks = []
    
            # Filter tracks based on selected artist and genre
            query = 'SELECT T_Name, Genre, Length, Username, T_ID FROM Track T, Create_T C WHERE T.T_ID = C.T_ID'
            
            params = []
    
            if username != 'All':
                query += ' AND C.Username = ?'
                params.append(username)
    
            if genre != 'All':
                query += ' AND T.genre = ?'
                params.append(genre)
    
            for row in cur.execute(query, params):
                t_name = row[0]
                genre = row[1]
                length = row[2]
                username = row[3]
                t_id = row[4]
                
                
                selected_tracks.append((t_name,genre,length,username))
    
            window['track'].update(values=selected_tracks)
    
        if event == 'Play':
            selected_track = values.get('track', [])
            t_id
            if not selected_track:  # empty list means False
                sg.popup('Please select a track.')
            else:
                cur.execute('SELECT MAX(H_ID) FROM Access')
                h_id = cur.fetchone()[0] + 1
                #print(h_id)
                
                cur.execute('SELECT T_ID FROM Track WHERE T_Name = ?',(selected_track[0][0],))
                t_id = cur.fetchone()[0]
                #print(t_id)
                selected_track = selected_track[0]
                timestamp = datetime.now()
                timestamp = timestamp.strftime('%d.%m.%Y %H:%M:%S')
                #print(timestamp)
                #history_P.insert(0, (selected_track, timestamp))
                cur.execute('INSERT INTO Access VALUES(?,?,?)', (int(h_id),UserName,int(t_id)))
                cur.execute('INSERT INTO History VALUES(?,?)', (int(h_id),timestamp))
                con.commit()
                text = 'Now Playing: ' + str(selected_track[0])
                sg.popup_timed(text, non_blocking=True, no_titlebar=True, auto_close_duration=1)
                
    
        if event == "Add to Playlist":
            window.close()
            Add_Playlist(UserName,t_id)
            
            
        if event == 'Back to Menu':
            window.close()
            Main_Page(UserName)
    
    window.close()
    
def Playlists(username):
    
    playlists = []
    for row in cur.execute('SELECT P_Name FROM Playlist'):
        playlists.append(row[0])
    selected_playlists = playlists
    layout = [
        [sg.Listbox(values=selected_playlists, size=(40, 7), key='playlist')],
        [sg.Button('Like', size=(12, 1)), sg.Button('Liked Playlists', size=(12, 1)), sg.Button('Back to Menu', size=(12, 1))]
    ]

    window = sg.Window('PlayLists Window', layout)
    liked_playlists = []  # Initialize an empty list to store liked playlists

    while True:
        event, values = window.read()

        if event == sg.WINDOW_CLOSED:
            break

        if event == 'Back to Menu':
            window.close()
            Main_Page(username)
        
        if event == 'Like':
            selected_playlists = values.get('playlist', [])
            if not selected_playlists:
                sg.popup('Please select a Playlist.')
            else:
                sg.popup("Playlist is Liked.")
                selected_playlists = values['playlist'][0]
                # Perform operations to like the playlist and update the liked_playlists list
                liked_playlists.append(selected_playlists)  # Add the liked playlist to the list

        if event == 'Liked Playlists':
            layout = [
                [sg.Text('Liked Playlist Window')],
                [sg.Listbox(values=liked_playlists, size=(25, 7), key='liked_playlists')],
            ]

            playlist_window = sg.Window('Liked Playlists', layout)
            while True:
                event, values = playlist_window.read()

                if event == sg.WINDOW_CLOSED:
                    break

                if event == 'Back to Playlists':
                    playlist_window.close()
                    break

            window.UnHide()
    
    window.close()

            
def Albums(username):
    # Get all albums
    albums = []
    for row in cur.execute('SELECT A_Name FROM Album'):
        albums.append(row[0])

    # Initialize selected albums
    selected_albums = albums

    # Create the layout for the GUI window
    layout = [
        [sg.Listbox(values=selected_albums, size=(25, 7), key='album')],
        [sg.Button('Select Album', size=(12, 1)), sg.Button('Back to Menu', size=(12, 1))]
    ]

    # Create the window
    window = sg.Window('Albums List Window', layout)

    while True:
        event, values = window.read()

        if event == sg.WINDOW_CLOSED:
            break

        if event == 'Back to Menu':
            window.close()
            Main_Page(username)

        if event == 'Select Album':
            selected_album = values.get('album', [])
            if not selected_album:
                sg.popup('Please select an album.')
            else:
                selected_album = values['album'][0]
                query = 'SELECT T.T_Name FROM Track T, Album A WHERE T.A_ID = A.A_ID AND A.A_Name = ?'
                tracks = []
                for row in cur.execute(query, (selected_album,)):
                    tracks.append(row[0])

                layout = [
                    [sg.Text(f'Tracks in Album "{selected_album}":')],
                    [sg.Listbox(values=tracks, size=(25, 7), key='track')],
                    [sg.Button('Play', size=(12, 1)), sg.Button('Back to Albums', size=(12, 1))]
                ]

                album_window = sg.Window(f'{selected_album} Tracks', layout)

                while True:
                    event, values = album_window.read()

                    if event == sg.WINDOW_CLOSED:
                        break

                    if event == 'Back to Albums':
                        album_window.close()
                        break

                    if event == 'Play':
                        selected_track = values.get('track', [])
                        if not selected_track:
                            sg.popup('Please select a track.')
                        else:
                            selected_track = values['track'][0]
                            timestamp = datetime.now()
                            history_P.insert(0,(selected_track, timestamp))
                            text = 'Now Playing: ' + str(selected_track)
                            sg.popup_timed(text, non_blocking=True, no_titlebar=True, auto_close_duration=1)
                            # Add your code to play the selected track here

    # Close the window and database connection
    window.close()   
    
def History(username):
     # Get the last tracks played
    history = []
    H_ID = []
    TS = []
    T_Name = []
    
    for row in cur.execute('SELECT H_ID, Timestamp FROM History'):
        H_ID.append(row[0])
        TS.append(row[1])
        
    for i in range(len(TS)):
        cur.execute("Select T_Name From Track T, Access A Where T.T_ID = A.T_ID AND H_ID = ?",(H_ID[i],))
        row = cur.fetchone()
        T_Name.append(row[0])
        
        
        history.append(f"You listened {T_Name[i]} at {TS[i]}")
    layout = [
        [sg.Text('Last Tracks Played', size=(20, 1))],
        [sg.Listbox(values= history, size=(50, 7), key='tracks')],
        [sg.Button('Back to Menu', size=(12, 1))]
    ]
    
    window = sg.Window('History Window', layout)
    
    while True:
        event, values = window.read()

        if event == sg.WINDOW_CLOSED:
            break
        if event == 'Back to Menu':
            window.close()
            Main_Page(username)
            
         
def Artist_Page(username):
    layout = [
        [sg.Text('Welcome, ' + username + '!')],
        [sg.Button('Check Track Plays')],
        [sg.Button("Artist's Tracks")],
        [sg.Button("Artist's Albums")],
        [sg.Button('Logout')],
    ]
    
    window = sg.Window('Artist Page', layout)
    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED,):
            break
        if event == "Check Track Plays":
            window.close()
            Check_Track_Plays(username)
        if event == "Artist's Tracks":
            window.close()
            Track_Window(username)
        if event == "Artist's Albums":
            window.close()
            Artist_Album(username)
        elif event == "Logout":
            window.close()
            Login_As()

    window.close()
    
def Artist_Album(username):
    Albums_A = []
    
    for row in cur.execute('SELECT A_Name FROM Album A, Create_A C WHERE C.A_ID = A.A_ID AND C.Username = ?', (username,)):
        Albums_A.append(row[0])

    layout = [
        [sg.Text('Albums')],
        [sg.Listbox(values=Albums_A, size=(30, 10), key='Album_A')],
        [sg.Button('Add Track'),sg.Button('Cretae Album'), sg.Button('Back to Menu')]
    ]

    window = sg.Window('Album', layout)

    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, 'Back'):
            break
        elif event == "Add Track":
            selected_album = values.get('Album_A', [])
            if not selected_album:  # Empty list means False
                sg.popup('Please select a album.')
            else:
               window.close() 
               Add_Track(username,selected_album)
        elif event == "Cretae Album":
               window.close() 
               Create_Album(username)
        elif event == "Back to Menu":
            window.close()
            Artist_Page(username)

    window.close()
    
def Create_Album(username):

    layout = [[sg.Text('Crete Album')],
              [sg.Text('Name: '),sg.Input(size=(30,1), key='album_name')],
              [sg.Button('Create'),sg.Button('Back to Album Page')]
              ]
    window =  sg.Window('Create Album Window', layout)
    
    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, 'Back'):
            break
        elif event == "Create":
            
            if values['album_name'] == '':
                sg.popup('Please enter name')   
            else:
                timestamp = datetime.now()
                timestamp = timestamp.strftime('%d.%m.%Y')
                counter = 1
                for row in cur.execute('SELECT * FROM Album'):
                    counter += 1 
                cur.execute('INSERT INTO Album VALUES (?,?,?)', (counter,values['album_name'],timestamp))
                con.commit()
                cur.execute('INSERT INTO Create_A VALUES (?,?)', (counter,username))
                con.commit()
                sg.popup('Successful')  
        elif event == "Back to Album Page":
            window.close()
            Artist_Album(username)
    window.close()  
    
def Add_Track(username,selected_album):
    Track_A = []

    for row in cur.execute('SELECT T_Name FROM Track T, Create_T C WHERE C.T_ID = T.T_ID AND C.Username = ? AND A_ID IS NULL ', (username,)):
        Track_A.append(row[0])

    layout = [
        [sg.Text('Select a track:')],
        [sg.Listbox(values=Track_A, size=(30, 10), key='Track_A')],
        [sg.Button('Add'), sg.Button('Back to Album Page')]
    ]

    window = sg.Window('Add Track', layout)

    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, 'Back'):
            break
        if event == "Add":
            selected_track = values['Track_A']
            if not selected_track:  # Empty list means False
                sg.popup('Please select a track.')
            else:
                selected_track = selected_track[0]
                cur.execute('SELECT T_ID FROM Track WHERE T_Name = ?',(selected_track,))
                t_id = cur.fetchone()[0]
                selected_album = selected_album[0]
                cur.execute('SELECT A_ID FROM Album WHERE A_Name = ?',(selected_album,))
                a_id = cur.fetchone()[0]
                cur.execute('UPDATE Track SET A_ID = ? WHERE T_ID = ?', (a_id,t_id))
                con.commit()
                sg.popup('Successful')
        elif event == "Back to Album Page":
            window.close()
            Artist_Album(username)

    window.close()
    
def Track_Window(username):
    Track_A = []

    for row in cur.execute('SELECT T_Name FROM Track T, Create_T C WHERE C.T_ID = T.T_ID AND C.Username = ?', (username,)):
        Track_A.append(row[0])

    layout = [
        [sg.Text('Select a track:')],
        [sg.Listbox(values=Track_A, size=(30, 10), key='Track_A')],
        [sg.Button('Check Plays'),sg.Button('Update'),sg.Button('Create Track'), sg.Button('Back to Menu')]
    ]

    window = sg.Window('Track Window', layout)

    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, 'Back'):
            break
        if event == "Check Plays":
            selected_track = values.get('Track_A', [])
            if not selected_track:  # Empty list means False
                sg.popup('Please select a track.')
            else:
                selected_track = selected_track[0]
                play_count, play_times = get_play_info(username, selected_track)
                if play_count > 0:
                    play_times_str = "\n".join(play_times)
                    sg.popup('The track "' + selected_track + '" has been played ' + str(play_count) + ' times in the last month.\n\nPlay times:\n' + play_times_str)
                else:
                    sg.popup('The track "' + selected_track + '" has not been played in the last month.')
        elif event == "Update":
            selected_track = values.get('Track_A', [])

            if not selected_track:  # Empty list means False
                sg.popup('Please select a track.')
            else:
                cur.execute("SELECT T_ID FROM Track WHERE T_Name = ?",(selected_track[0],))
                t_id = int(cur.fetchone()[0])
                window.close() 
                update_page(username,t_id)
        elif event == "Create Track":
            window.close()
            Create_Track(username)
        elif event == "Back to Menu":
            window.close()
            Artist_Page(username)

    window.close()
    
def Create_Track(username):
    albums = []
    
    for row in cur.execute('SELECT A_Name FROM Album A'):
        albums.append(row[0])
    artist = []
    for row in cur.execute('SELECT Username FROM Artist A'):
        artist.append(row[0])
    layout = [[sg.Text('Crete Track')],
              [sg.Text('Name: ',size=(20, 1)),sg.Input(size=(30,1), key='newtrack_name')],
              [sg.Text('Genre: ',size=(20, 1)),sg.Input(size=(30,1), key='newtrack_genre')],
              [sg.Text('Length: ',size=(20, 1)),sg.Input(size=(30,1), key='newtrack_length')],
              [sg.Text('Collaborative Artist: ',size=(20, 1)),sg.Combo(artist, size=(40,10), key='newtrack_colartist')],
              [sg.Text('Album: ',size=(20, 1)),sg.Combo(albums, size=(40,10), key='album')],             
              [sg.Button('Create')],
              [sg.Button('Back to Track Page')]]
    window =  sg.Window('Create Track Window', layout)
    timestamp = datetime.now()
    timestamp = timestamp.strftime('%d.%m.%Y')
    while True:
        event, values = window.read()
        numbers = ['0','1','2','3','4','5','6','7','8','9']
        dot = [':','.']
        if event in (sg.WINDOW_CLOSED, 'Back'):
            break
        elif event == 'Create':
            if values['newtrack_name'] == '':
                sg.popup('Please enter name')
            elif values['newtrack_genre'] == '':
                sg.popup('Please enter genre')  
            elif values['newtrack_length'] == '':
                sg.popup('Please enter length')
            elif len(values['newtrack_length'])!=4 and len(values['newtrack_length'])!=5:
                sg.popup('Please enter valid length') 
            else:
                if values['newtrack_length'][-1] not in numbers or values['newtrack_length'][-2] not in numbers or values['newtrack_length'][-3] not in dot or values['newtrack_length'][-4] not in numbers  or int(values['newtrack_length'][-2]) > 5:                 
                    sg.popup('Please enter valid length')         
                else:
                    counter = 1
                    for row in cur.execute('SELECT * FROM Track'):
                        counter += 1 
                    
                    a_id = cur.execute('SELECT A_ID FROM Album A WHERE A_Name=?',(values['album'],))
                    try:
                        a_id=a_id.fetchone()[0]
                        cur.execute('INSERT INTO Track VALUES (?,?,?,?,?,?)', (counter,a_id,values['newtrack_name'],values['newtrack_genre'],(values['newtrack_length']),timestamp))
                        con.commit()
                        sg.popup('Successful')
                        
                    except:
                        cur.execute('INSERT INTO Track VALUES (?,?,?,?,?,?)', (counter,None,values['newtrack_name'],values['newtrack_genre'],(values['newtrack_length']),timestamp))
                        con.commit()
                        sg.popup('Successful')
                    cur.execute('INSERT INTO Create_T VALUES (?,?)', (counter,username))
                    con.commit()                
                    if not values['newtrack_colartist'] == '':
                        cur.execute('INSERT INTO Create_T VALUES (?,?)', (counter,values['newtrack_colartist']))
                        con.commit()
        elif event == "Back to Track Page":
            window.close()
            Track_Window(username)
    window.close()
    
def update_page(username,trackid):
     info = ['Track Name','Genre']
     
     layout = [[sg.Text('Select Information to Edit ')],
               [sg.Combo(info, size=(40,10), key='info')],
               [sg.Input(size=(30,1), key='new_value')],
               [sg.Button('Update')],
               [sg.Button('Back to Track Page')]]
     window =  sg.Window('Track Update Window', layout)
     
     while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, 'Back'):
            break
        elif event =='Update':
            if values['new_value'] == '':
                sg.popup('Please enter new information')
            elif values['info'] == '':
                sg.popup('Please select information')
            else:
                if values['info']=='Track Name':
                    
                    cur.execute('UPDATE Track SET T_Name = ? WHERE T_ID = ?', (values['new_value'],trackid))
                    con.commit()
                    sg.popup('Successful')
                elif values['info']=='Genre':

                    cur.execute('UPDATE Track SET Genre = ? WHERE T_ID = ?', (values['new_value'],trackid))
                    con.commit()
                    sg.popup('Successful')
        elif event =='Back to Track Page':
             window.close()
             Track_Window(username)
     window.close()


def Check_Track_Plays(username):
    Track_A = []

    for row in cur.execute('SELECT T_Name FROM Track T, Create_T C WHERE C.T_ID = T.T_ID AND C.Username = ?', (username,)):
        Track_A.append(row[0])

    layout = [
        [sg.Text('Select a track:')],
        [sg.Listbox(values=Track_A, size=(30, 10), key='Track_A')],
        [sg.Button('Check Plays'), sg.Button('Back to Menu')]
    ]

    window = sg.Window('Track Plays', layout)

    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, 'Back'):
            break
        if event == "Check Plays":
            selected_track = values.get('Track_A', [])
            if not selected_track:  # Empty list means False
                sg.popup('Please select a track.')
            else:
                selected_track = selected_track[0]
                play_count, play_times = get_play_info(username, selected_track)
                if play_count > 0:
                    play_times_str = "\n".join(play_times)
                    sg.popup('The track "' + selected_track + '" has been played ' + str(play_count) + ' times in the last month.\n\nPlay times:\n' + play_times_str)
                else:
                    sg.popup('The track "' + selected_track + '" has not been played in the last month.')

        elif event == "Back to Menu":
            window.close()
            Artist_Page(username)

    window.close()

def get_play_info(username, track_name):
    play_count = 0
    play_times = []

    # Calculate the start date for the last month
    last_month = datetime.now() - timedelta(days=30)

    # Iterate over the history_P list and count plays in the last month
    for track, timestamp in history_P:
        if track == track_name and timestamp >= last_month:
            play_count += 1
            play_times.append(str(timestamp))

    return play_count, play_times
def Add_Playlist(UserName, T_ID):
    layout = [[sg.Text("Pick the playlist type you want to add.")],
              [sg.Button("Add to Private Playlist"),sg.Button("Add to Public Playlist")],
              [sg.Button("Create a Playlist")],
              [sg.Button("Cancel")]     
             ]
    
    window = sg.Window("Playlist Selection", layout)
    
    while True:
        event, values = window.read()
    
        if event == "Add to Private Playlist":
            window.close()
            Private(UserName,T_ID)
        elif event == "Add to Public Playlist":
            window.close()
            Public(UserName,T_ID)
        elif event == "Create a Playlist":
            window.close()
            Create_Playlist(UserName, T_ID)
        elif event == "Cancel":
            window.close()
            Tracks(UserName)
        elif event == sg.WIN_CLOSED:
            break

    window.close
 
def Private(UserName,T_ID):
    playlists = []
    
    for row in cur.execute('SELECT P_Name FROM Has_p H, Playlist P WHERE Username = ? AND H.P_ID = P.P_ID AND Visibility = 0',(UserName,)):
        playlists.append(row[0])
        
    layout = [[sg.Listbox(playlists, key = "playlist")],[sg.Button("Add to Playlist")],
              [sg.Button("Back")]]
    window = sg.Window("Private Playlists", layout)
    
    while True:
        event, values = window.read()
        if event == "Add to Playlist":
           playlist_name = values["playlist"]
           cur.execute("SELECT T_Name FROM Track WHERE T_ID = ?",(T_ID,))
           t_name = cur.fetchone()[0]
           if playlist_name == []:
               text = 'Please select a playlist first!'
               sg.popup(text)
           
           else:
               cur.execute("SELECT P_ID FROM Playlist P WHERE P_Name = ?", (playlist_name[0],))
               playlist_id = cur.fetchone()[0]
               
               cur.execute("SELECT P_ID, T_ID, Username FROM Has_p WHERE T_ID = ? AND P_ID = ? AND Username = ?",(T_ID,playlist_id,UserName))
               row = cur.fetchone()
               print(row)
               if row is None:
                   cur.execute('INSERT INTO Has_p VALUES (?,?,?)',(T_ID,playlist_id,UserName))
                   con.commit()
                   
                   text = (t_name,"is successfully added to",playlist_name)
                   sg.popup_timed(text, non_blocking=True, no_titlebar=True, auto_close_duration=1)
               else:
                   text = 'This track is already in this list!'
                   sg.popup(text)
            
        elif event == "Back":
            window.close()
            Add_Playlist(UserName, T_ID)
        
        elif event == sg.WIN_CLOSED:
            break
        
    window.close()
    
    
def Public(UserName,T_ID):
    playlists = []
    
    for row in cur.execute('SELECT P_Name FROM Has_p H, Playlist P WHERE Username = ? AND H.P_ID = P.P_ID AND Visibility = 1',(UserName,)):
        playlists.append(row[0])
        
    layout = [[sg.Listbox(playlists, key = "playlist")],[sg.Button("Add to Playlist")],
              [sg.Button("Back")]
             ]
    window = sg.Window("Private Playlists", layout)
    
    while True:
        event, values = window.read()
        if event == "Add to Playlist":
           playlist_name = values["playlist"]
           
           if playlist_name == []:
               text = 'Please select a playlist first!'
               sg.popup(text)
           
           else:
               cur.execute("SELECT P_ID FROM Playlist P WHERE P_Name = ?", (playlist_name[0],))
               playlist_id = cur.fetchone()[0]
                
               
               cur.execute("SELECT P_ID, T_ID, Username FROM Has_p WHERE T_ID = ? AND P_ID = ? AND Username = ?",(T_ID,playlist_id,UserName))
               row = cur.fetchone()

               
               if row is None:
                   cur.execute('INSERT INTO Has_p VALUES (?,?,?)',(T_ID,playlist_id,UserName))
                   con.commit()
                   
               else:
                   text = 'This track is already in this list!'
                   sg.popup(text)
            
        elif event == "Back":
            window.close()
            Add_Playlist(UserName,T_ID)               
                
            #bitmedi
            
        elif event == sg.WIN_CLOSED:
            break
    
    window.close()    
    
def Create_Playlist(UserName, T_ID):
    
    layout = [[sg.Text("Playlist Name: "), sg.Input(size=(20,1), key ="playlist_name")],
              [sg.Text("Playlist Type: "), sg.Radio('Public', 'radio', default=True, key='public'),
              sg.Radio('Private', 'radio', key='private')],
              [sg.Button("Create Playlist"),sg.Button("Cancel")]
              ]

    window = sg.Window("Create Playlist", layout)
    
    while True:
        event, values = window.read()
        
        if event == "Create Playlist":
            playlist_name = values["playlist_name"]
            playlist_type = 0 if values["private"] else 1

            if playlist_name == "":
                sg.popup("Playlist name cannot be empty!")
            else:
                cur.execute('SELECT MAX(P_ID) FROM Playlist')
                p_id = cur.fetchone()[0] + 1
                print(p_id)
                cur.execute('INSERT INTO Has_p VALUES(?,?,?)', (int(T_ID),int(p_id),UserName))
                cur.execute('INSERT INTO Playlist VALUES(?,?,?)', (int(p_id),playlist_name,playlist_type))
                con.commit()
                text = "Playlist is successfully created!"
                sg.popup_timed(text, non_blocking=True, no_titlebar=True, auto_close_duration=1)
                print(p_id)    
                
        elif event == "Cancel":
            window.close()
            Add_Playlist(UserName, T_ID)
        elif event == sg.WINDOW_CLOSED:
            break

    window.close()


def Search_Users(username):
    layout = [
        [sg.Text('Search for Standard Users')],
        [sg.Text('Username:', size=(10, 1)), sg.Input(key='search_username')],
        [sg.Button('Search'), sg.Button('Back to Menu')]
    ]

    window = sg.Window('Search Users', layout)
    while True:
        event, values = window.read()
        if event == sg.WINDOW_CLOSED:
            break
        if event == 'Back to Menu':
            window.close()
            Main_Page(username)
        if event == 'Search':
            search_username = values['search_username']
            if not search_username:
                sg.popup('Please enter a username to search.')
            else:
                cur.execute('SELECT Username FROM Standard WHERE Username = ?', (search_username,))
                row = cur.fetchone()
                if row is not None:
                    if row[0] == username:
                        sg.popup('You cannot follow yourself: ' + search_username)
                    else:
                        cur.execute('SELECT Follower_ID FROM Follow WHERE Follower_ID = ? AND Followee_ID = ?', (username, search_username))
                        followee = cur.fetchone()
                        if followee is not None:
                            sg.popup(f'You are already following user: {search_username}')
                        else:
                            cur.execute('INSERT INTO Follow VALUES (?, ?)', (username, search_username))
                            con.commit()
                            sg.popup(f'You are now following user: {search_username}')
                else:
                    sg.popup('User not found: ' + search_username)

    window.close()
    con.close()

def Follower_playlist(username):
  
    
    layout = [
        [sg.Button("Follow Back", size=(15,1))],
        [sg.Button('Followers Playlists', size=(15,1))],
    ]
    window = sg.Window('Follower Playlist', layout)
    
    while True:
        event, values = window.read()
        
        if event == sg.WINDOW_CLOSED:
            break
        
        if event == "Follow Back":
            cur.execute('SELECT Followee_ID FROM Follow WHERE Follower_ID = ?', (username,))
            followees = cur.fetchall()
            followee_list = [f[0] for f in followees]
            cur.execute('SELECT Follower_ID FROM Follow WHERE Followee_ID = ?', (username,))
            followers = cur.fetchall()
            follower_list = [f[0] for f in followers]
            followee_list.extend(follower_list)
            sg.popup('Followed Back :)')
            con.commit()
            
        if event == 'Followers Playlists':
            cur.execute('SELECT P.P_Name FROM Playlist P JOIN Has_p H ON P.P_ID = H.P_ID JOIN Standard S ON S.Username = H.Username JOIN Follow F ON S.Follower_ID = F.Follower_ID WHERE P.Visibility = 1 AND F.Followee_ID = ?', (username,))
            playlists = cur.fetchall()
            selected_playlists = [p[0] for p in playlists]
            
            layout = [
                [sg.Text('Followers Playlist Window')],
                [sg.Listbox(values=selected_playlists, size=(25, 7), key='selected_playlists')],
            ]

            playlist_window = sg.Window('Followers Playlists', layout)
            
            while True:
                event, values = playlist_window.read()
                if event == sg.WINDOW_CLOSED:
                    break

            playlist_window.close()

    window.close()
   
def Open_Profile(username):
    layout = [
        [sg.Button("My Playlists",size=(15,1))],
        [sg.Button('View Followers',size=(15,1))],
        [sg.Button('View Followees',size=(15,1))],
        [sg.Button('Back to Menu',size=(15,1))]
    ]
    window = sg.Window('Open Profile', layout)
    while True:
        event, values = window.read()
        if event == sg.WINDOW_CLOSED:
            break
        if event == "My Playlists":
            window.close()
            My_Playlists(username)
        if event == 'Back to Menu':
            window.close()
            Main_Page(username)
        if event == 'View Followers':
            cur.execute('SELECT Follower_ID FROM Follow Where Followee_ID = ?', (username,))
            followers = cur.fetchall()
            follower_list = [f[0] for f in followers]
            sg.popup(f'Followers of {username}:  {", ".join(follower_list)}')
            Follower_playlist(username)
        if event == 'View Followees':
            cur.execute('SELECT  Followee_ID FROM Follow Where Follower_ID = ?', (username,))
            followees = cur.fetchall()
            followee_list = [f[0] for f in followees]
                    
            sg.popup(f'Followees of {username} :  {", ".join(followee_list)}')

    window.close()
    
def My_Playlists(UserName):
    P_ID = []
    P_Name = []
    
    for row in cur.execute('SELECT P_ID FROM Has_p WHERE Username = ?',(UserName,)):
        P_ID.append(row[0])
    
    for i in range(len(P_ID)):
        cur.execute("Select P_Name From Playlist P, Has_p HP Where P.P_ID = ?",(P_ID[i],))
        row = cur.fetchone()
        if row:
            P_Name.append(row[0])
    
    
    
    layout = [
        [sg.Text('My Playlists', size=(20, 1))],
        [sg.Listbox(values= P_Name, size=(50, 7), key='albums')],
        [sg.Button('Back to Profile', size=(12, 1))]
    ]
    
    window = sg.Window('Profile Page', layout)
    
    while True:
        event, values = window.read()

        if event == sg.WINDOW_CLOSED:
            break
        if event == 'Back to Profile':
            window.close()
            Open_Profile(UserName)
            
            
            

Login_As()

